> start minie server:
```shell
mvn clean compile exec:java
```

> call example:
```shell
curl 'http://localhost:8080/minie/query' -X POST -d 'Obama visited the white house.' | jq .
```
                                                Post                data                   json

In [1]:
import requests

url = 'http://localhost:8080/minie/query'
data = 'Obama visited the white house.'
r = requests.post(url, data=data)
print(r.json())

{'facts': [{'subject': 'Obama', 'predicate': 'visited', 'object': 'white house'}]}


> MinIE.Mode.SAFE  
> Change mode in file FaceResource.java line 30:
```java
MinIE minie = new MinIE(sentence, FactsResource.parser, MinIE.Mode.SAFE);
```

In [7]:
import pickle
import pandas as pd
import spacy
from tqdm import tqdm
url = 'http://localhost:8080/minie/query'
nlp = spacy.load('en_core_web_lg')

asd = None
def prepare_text(text):
    global asd
    doc=nlp(text)
    result=[]
    for sent in list(doc.sents):
            sent=str(sent)
            sent = sent.rstrip() #remove end space
            r = requests.post(url, data=sent)
            try:
                print(r.json())
            except:
                asd = [sent,r]
            for facts in r.json()['facts']:
                result.append({'subject':facts['subject'],'relation': facts['predicate'],'object':facts['object']})
    return result

def process(file_name):
    results = []
    var = pickle.load(open('./cleaned_data_split/'+file_name+'.sav', 'rb'))
    for text in tqdm(var):
        results += prepare_text(text)
    result = pd.DataFrame(results)
    result.to_csv('./result_minie/'+file_name+'r.csv',index=False)

In [8]:
process('a0') # a1 a2 a3 ...

  0%|                                                                                        | 0/16042 [00:00<?, ?it/s]

{'facts': [{'subject': 'retrospective chart review', 'predicate': 'describes', 'object': 'epidemiology of QUANT_O_1 patients with culture-proven mycoplasma pneumoniae infections at king abdulaziz university hospital'}, {'subject': 'retrospective chart review', 'predicate': 'describes', 'object': 'clinical features'}, {'subject': 'retrospective chart review', 'predicate': 'describes', 'object': 'epidemiology of QUANT_O_1 patients with culture-proven mycoplasma pneumoniae infections at jeddah'}, {'subject': 'king abdulaziz university hospital', 'predicate': 'is', 'object': 'saudi arabia'}, {'subject': 'jeddah', 'predicate': 'is', 'object': 'saudi arabia'}]}
{'facts': [{'subject': 'patients with positive m. pneumoniae cultures from respiratory specimens from january 1997 through december 1998', 'predicate': 'were identified through', 'object': 'microbiology records'}]}


  0%|                                                                                        | 0/16042 [00:06<?, ?it/s]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
asd

['charts of patients were reviewed.', <Response [500]>]

In [10]:
r = requests.post(url, data=asd[0])

In [11]:
r

<Response [500]>

> TODO: Find out why 'charts of patients were reviewed.' cause Response [500]